# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

# **Hyperparameter Tunning**

In [2]:
num_epochs = 50
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

# **Build Model**

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# **Data Preprocess**

In [5]:
train_paths = glob('D:/dataset/cifar/train/*.png')[:1000]
test_paths = glob('D:/dataset/cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# **Checkpoint**

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

#history 하기전에 모델에 변수로 저장해야한다.
history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
31/31 [==============================] - 4s 118ms/step - loss: 2.3188 - accuracy: 0.0981 - val_loss: 2.2993 - val_accuracy: 0.0958
Epoch 2/50
31/31 [==============================] - 3s 105ms/step - loss: 2.2981 - accuracy: 0.1240 - val_loss: 2.2982 - val_accuracy: 0.1048
Epoch 3/50
31/31 [==============================] - 3s 111ms/step - loss: 2.2600 - accuracy: 0.1529 - val_loss: 2.2254 - val_accuracy: 0.1431
Epoch 4/50
31/31 [==============================] - 4s 114ms/step - loss: 2.2342 - accuracy: 0.1539 - val_loss: 2.2247 - val_accuracy: 0.1502
Epoch 5/50
31/31 [==============================] - 4s 118ms/step - loss: 2.1677 - accuracy: 0.2035 - val_loss: 2.1118 - val_accuracy: 0.2198
Epoch 6/50
31/31 [==============================] - 3s 105ms/step - loss: 2.0904 - accuracy: 0.2252 - val_loss: 2.0243 - val_accuracy: 0.2278
Epoch 7/50
31/31 [==============================] - 3s 111ms/step - loss

# **Predict**

## **이미지를 Load 직접 load해서 넣는 방법**

In [15]:
path = test_paths[0]
path

'D:/dataset/cifar/test\\0_cat.png'

In [16]:
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile, dtype = tf.float32)
image.shape

TensorShape([32, 32, 3])

In [19]:
image = image[tf.newaxis, ...]

In [20]:
image.shape

TensorShape([1, 32, 32, 3])

In [22]:
pred = model.predict(image)

In [23]:
pred.shape

(1, 10)

In [24]:
pred

array([[0.01223146, 0.0098235 , 0.07381397, 0.2569962 , 0.19544458,
        0.18445283, 0.05432431, 0.18798509, 0.00885634, 0.01607163]],
      dtype=float32)

In [25]:
np.argmax(pred)

3

## **generator에서 데이터를 가져오는 방법**

In [26]:
test_image, test_label = next(iter(test_dataset))

In [27]:
test_image.shape

TensorShape([32, 32, 32, 3])

In [28]:
pred = model.predict(test_image)

In [29]:
pred.shape

(32, 10)

In [30]:
pred[0]

array([0.01223146, 0.0098235 , 0.07381397, 0.2569962 , 0.19544458,
       0.18445285, 0.0543243 , 0.18798509, 0.00885634, 0.01607163],
      dtype=float32)

In [31]:
np.argmax(pred[0])

3

## **generator에 넣는 방법**

In [34]:
pred = model.predict_generator(test_dataset.take(2))
# test_dataset.take(1) 한 배치만 가져온다. 
#전체 이미지를 넣을수 없어서 take를 사용해서 배치마다 넣어겨 결과값을 가져온ㄷ,

In [35]:
pred.shape

(64, 10)

In [36]:
image, label = next(iter(test_dataset))

In [37]:
image.shape

TensorShape([32, 32, 32, 3])

In [38]:
evals = model.evaluate(image, label)

1/1 [==============================] - 0s 1ms/step - loss: 1.6542 - accuracy: 0.5000


In [39]:
evals

[1.6541881561279297, 0.5]